# HTS [Part 3]

In [1]:
%pylab inline
import sys
from ipywidgets import interact

Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('src')
from motorControl import *
from trajectoryPlanning import *
from util import *

## Load Absolute Home, Upper right corner, out of the way, aligned with sharpie marks
f = open('savedPositions/absolute/home.p', 'r')
home = pickle.load(f)
f.close()

In [4]:
rc = connect(portName = "/dev/tty.usbserial-A9ETDN3N")

### Initialized Motor Classes

- Really important to initialze these first to restore positions from last session!

In [5]:
#Initialize motor objects for each motor:
vL = Motor(address = 0x81, motorNumber = 2, rc = rc, signFlipped = False, \
           motorCounter = 0, kPID = [1e-2, 1.0])
vR = Motor(address = 0x81, motorNumber = 1, rc = rc, signFlipped = False, \
           motorCounter = 1, kPID = [1e-2, 1.0])
LR = Motor(address = 0x80, motorNumber = 1, rc = rc, signFlipped = False, 
           motorCounter = 2, kPID = [1e-2, 1.0])
FB = Motor(address = 0x80, motorNumber = 2, rc = rc, signFlipped = True, \
           motorCounter = 3, kPID = [1e-2, 1.0])
yaw = Motor(address = 0x82, motorNumber = 2, rc = rc, signFlipped = False, \
            motorCounter = 4, kPID = [1e-2, 1.0])
pitch = Motor(address = 0x82, motorNumber = 1, rc = rc, signFlipped = False, \
              motorCounter = 5, kPID = [1e-2, 1.0])

#Keep in a nice motor list:
motors = [vL, vR, LR, FB, yaw, pitch]

In [6]:
getPositions(motors)

[0, 0, 0, 0, 0, 0]

In [1]:
interact(manualControl, leftUD = (-50, 50), rightUD = (-50, 50), leftRight = (-50, 50), \
         fB = (-50, 50), tilt = (-50, 50), pan = (-50, 50))

NameError: name 'interact' is not defined

In [11]:
stopAll(rc)
savePositions(motors)
getPositions(motors)

[0, 0, 0, 0, 0, 0]

In [10]:
levelRig(motors, home, rc, moveTime = 5.0)

Rig is not level!, difference from home = 279


KeyboardInterrupt: 

## Save absolute home

- Only needed for hard resets

In [28]:
#Upper right corner, out of the way, aligned with sharpie marks
# f = open('savedPositions/absolute/home.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Save Relative Position

In [47]:
# delta = np.array(getPositions(motors))-np.array(home)
# #Position A, directly overhead in center, pointing down
# f = open('savedPositions/relative/HTS/A.p', 'wb')
# pickle.dump(delta, f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Load Relative Positions

In [9]:
#Position 1, directly overhead in center, pointing down
f = open('savedPositions/relative/HTS/A.p', 'r')
A = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

## Move the rig

In [10]:
targetPositions = A
totalTime = 45.0
rampTime = 5.0

startingPositions = getPositions(motors)
lookAheadTime = 1.0
tolerance = 100.0 #Anything less than this many ticks we're calling "Not a move"

motorsToMove = []
targetPositionsToMove = []
for i, motor in enumerate(motors):
    if abs(motor.getPosition()-targetPositions[i]) > tolerance:
        motorsToMove.append(motor)
        targetPositionsToMove.append(targetPositions[i])
        
trajectories = []
for i, motor in enumerate(motorsToMove):
    trajectories.append(SimpleQuadraticTrajectory(tu = rampTime, tt = totalTime, \
                                            p1 = motor.getPosition(), p2 = targetPositionsToMove[i]))
    
print np.array(targetPositions) - startingPositions
print len(motorsToMove)

[ 29753 -29721      9     -1      0     98]
2


In [11]:
for motor in motorsToMove:
    motor.initialize(targetVelocityMin = -2500.0, targetVelocityMax = 2500.0)
    motor.clearTracking()
    
startTime = time.time()
timeElapsed = 0.0
    
while timeElapsed < totalTime:
    timeElapsed = time.time()-startTime
    
    for i, motor in enumerate(motorsToMove):
        lookAheadValue = trajectories[i].compute(timeElapsed + lookAheadTime)
        motor.controlledMove(targetPosition = lookAheadValue, timeToReach = lookAheadTime)

stopAll(rc)
savePositions(motors)
print getPositions(motors), targetPositions
print getPositions(motors) - np.array(targetPositions)

[-33196, 32917, -123522, 94408, 30897, 44517] [ -33124   32845 -123513   94407   30897   44615]
[-72  72  -9   1   0 -98]


In [10]:
stopAll(rc)
savePositions(motors)